# 🚉 Índice Logístico de accesibilidad a SITVA
Este notebook calcula el tiempo de acceso desde cada lote (poligono) a las estaciones del SITVA (Sistema Integrado de Transporte del Valle de Aburrá) y luego construye un índice ponderado por línea usando una función logística para penalizar los tiempos mayores.

# 1. Importación de librerías necesarias

In [ ]:
import geopandas as gpd
import pandas as pd
import networkx as nx
from shapely.geometry import Point
from scipy.spatial import cKDTree

# 2. Configuración inicial

In [ ]:
# Rutas
ruta_base = "../data"
ruta_salida = "../output"
archivo_lotes = ruta_base + "Lotes.shp"
archivo_estaciones = ruta_base + "sitva_estaciones.gpkg"
archivo_nodes = ruta_base + "nodes.gpkg"
archivo_edges = ruta_base + "edges.gpkg"

# 3. Carga de archivos

In [ ]:
# Cargar geodatos
gdf_lotes = gpd.read_file(archivo_lotes)
gdf_estaciones = gpd.read_file(archivo_estaciones)
gdf_nodes = gpd.read_file(archivo_nodes)
gdf_edges = gpd.read_file(archivo_edges)

# 4. Proyección a coordenadas métricas (UTM)

In [ ]:
# Reproyección a UTM y corrección geometrías
crs_utm = 32618
gdf_lotes = gdf_lotes.to_crs(crs_utm)
gdf_nodes = gdf_nodes.to_crs(crs_utm)
gdf_edges = gdf_edges.to_crs(crs_utm)
gdf_estaciones = gdf_estaciones.to_crs(crs_utm)
gdf_lotes["geometry"] = gdf_lotes["geometry"].buffer(0)

# 5. Filtro de estaciones SITVA dentro del rango de análisis.

In [ ]:
# Asignar nodos más cercanos a lotes y estaciones
gdf_lotes["centroide"] = gdf_lotes.geometry.centroid
tree = cKDTree(list(zip(gdf_nodes.geometry.x, gdf_nodes.geometry.y)))
gdf_lotes["nodo_lote"] = gdf_nodes.iloc[tree.query(list(zip(gdf_lotes["centroide"].x, gdf_lotes["centroide"].y)))[1]]["osmid"].values

contorno_lotes = gdf_lotes.unary_union.convex_hull
estaciones_dentro = gdf_estaciones[gdf_estaciones.within(contorno_lotes)]
coords_estaciones = list(zip(estaciones_dentro.geometry.x, estaciones_dentro.geometry.y))
idx_estaciones = tree.query(coords_estaciones)[1]
estaciones_dentro["nodo_estacion"] = gdf_nodes.iloc[idx_estaciones]["osmid"].values

# 6. Construcción de red dirigida con tiempos peatonales

In [ ]:
# Crear grafo dirigido con pesos en tiempo
G = nx.DiGraph()
for _, row in gdf_edges.iterrows():
    G.add_edge(row["u"], row["v"], weight=row["tiempo_min_peaton"])

# 7. Crear columnas tiempos

In [ ]:
# Mapeo de estaciones a columnas
estacion_column_map = {}
for _, row in estaciones_dentro.iterrows():
    nombre = row["nombre"].replace(" ", "_")
    linea = row["linea"].replace(" ", "_")
    columna = f"{nombre}_{linea}"
    estacion_column_map[columna] = row["nodo_estacion"]

# 8. Calcular tiempos desde cada lote a cada estación SITVA.

In [ ]:
# Calcular tiempos a estaciones desde cada lote
def calcular_tiempos(nodo_lote):
    try:
        tiempos = nx.single_source_dijkstra_path_length(G, nodo_lote, weight="weight")
        return {nombre_col: tiempos.get(nodo_est, None) for nombre_col, nodo_est in estacion_column_map.items()}
    except:
        return {nombre_col: None for nombre_col in estacion_column_map.keys()}

df_tiempos = gdf_lotes["nodo_lote"].apply(calcular_tiempos).apply(pd.Series)

# 9. Unir tiempos y lotes

In [ ]:
# Unir tiempos con los lotes
proximidad = pd.concat([gdf_lotes.drop(columns=["centroide"]), df_tiempos], axis=1)

# 10. Calcular Índice

In [ ]:
# Calcular índice logístico SITVA
import numpy as np

columnas_tiempo = [col for col in proximidad.columns if "Línea" in col or any(linea in col for linea in ["A", "B", "J", "K", "L", "M", "H", "P", "T", "1", "2"])]

pesos_linea = {
    "A": 1.00, "B": 0.39, "K": 0.07, "J": 0.07, "H": 0.04, "M": 0.06,
    "P": 0.09, "1": 0.07, "2": 0.03, "T": 0.09, "T-A": 0.09, "L": 0.01
}

# Pesos por estación completa
pesos_estaciones = {}
for col in columnas_tiempo:
    linea = col.split("_")[-1]
    if linea in pesos_linea:
        pesos_estaciones[col] = pesos_linea[linea]
    else:
        print(f"[ADVERTENCIA] Línea no encontrada en diccionario: {linea}")

def calcular_indice_logistico(fila):
    total = 0
    for col in columnas_tiempo:
        tiempo = fila[col]
        if pd.notna(tiempo):
            peso_base = pesos_estaciones.get(col, 0)
            penalizacion = 1 / (1 + np.exp((tiempo - 12)/3))
            total += peso_base * penalizacion
    return total

proximidad["tiempo_min"] = proximidad[columnas_tiempo].min(axis=1)
proximidad["estacion_mas_cercana"] = proximidad[columnas_tiempo].idxmin(axis=1)
proximidad["indice_accesibilidad_SITVA"] = proximidad.apply(calcular_indice_logistico, axis=1)

In [ ]:
# Asegurarse de que el DataFrame sea un GeoDataFrame y tenga geometría válida
proximidad = gpd.GeoDataFrame(proximidad, geometry="geometry", crs=gdf_lotes.crs)

# Ruta de salida
salida_final = ruta_salida + "lotes_accesibilidad_sitva.gpkg"

# Guardar en GeoPackage
proximidad.to_file(salida_final, layer="lotes", driver="GPKG")

print(f"✅ Archivo exportado exitosamente a: {salida_final}")
